# Resampling Methods

## Lab: Cross-Validation and the Bootstrap

The Validation Set Approach

In [1]:
#Resampling Methods
# 5.3 Lab: Cross-Validation and the Bootstrap
library(ISLR)
set.seed(1)
train=sample(392,196) #training for cross validation, choosing 196 random samples from 392 observation


Warning message:
"package 'ISLR' was built under R version 3.4.2"

In [3]:
attach(Auto)
fix(Auto)
lm.fit=lm(mpg~horsepower,data=Auto, subset=train)
mean((mpg-predict(lm.fit,Auto))[-train]^2) # Mean Test Error is 26.14

The following objects are masked from Auto (pos = 3):

    acceleration, cylinders, displacement, horsepower, mpg, name,
    origin, weight, year



[1] 26.14142

In [4]:
lm.fit2=lm(mpg∼poly(horsepower,2) ,data=Auto , subset =train )
mean((mpg-predict(lm.fit2,Auto ))[-train ]^2) #MSE=19.82 lower than linear model

[1] 19.82259

In [5]:
lm.fit3=lm(mpg∼poly(horsepower,3) ,data=Auto , subset =train )
mean((mpg-predict(lm.fit3,Auto ))[-train ]^2) # MSE=19.78 lower than quadratic model


[1] 19.78252

#validation set error rates for the models with linear,quadratic, and cubic terms are 26.14, 19.82, and 19.78, respectively.

In [6]:
set.seed(2)
train=sample(392,196) #taking another random sample 
lm.fit=lm(mpg∼horsepower , subset=train ) #fitting linear model
mean((mpg-predict(lm.fit,Auto ))[-train ]^2) #MSE for linear model=23.3

[1] 23.29559

In [9]:
lm.fit2=lm(mpg∼poly(horsepower,2) , data=Auto,subset=train )
mean((mpg-predict(lm.fit2,Auto ))[-train ]^2) #MSE=18.9 

[1] 18.90124

In [10]:
lm.fit3=lm(mpg∼poly(horsepower,3) ,data=Auto , subset =train )
mean((mpg-predict(lm.fit3,Auto ))[-train ]^2)


[1] 19.2574

#validation set error rates for the models with linear,quadratic, and cubic terms are 23.30, 18.90, and 19.26, respectively. It shows quadraic model is better than linear and cubic model

## LOOCV

LOOCV estimate can be automatically computed for any generalized
linear model using the glm() and cv.glm() functions. if we use glm() to fit a model
without passing in the family argument, then it performs linear regression,
just like the lm() function. 

In [10]:
#5.3.2 Leave-One-Out Cross-Validation
# Linear model - using glm without family argument
glm.fit =glm(mpg∼horsepower,data= Auto)
coef(glm.fit )
lm.fit=lm(mpg~horsepower, data=Auto)
coef(lm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [12]:
# cv.glm(). The cv.glm() function is part of the boot library.
library(boot)
glm.fit =glm(mpg∼horsepower,data= Auto)
cv.err =cv.glm(Auto,glm.fit) # LOOCV cross validation error
names(cv.err)
summary(cv.err)
cv.err$delta 
#The two numbers in the delta vector contain the cross-validation results.

[1] "call"  "K"     "delta" "seed"

      Length Class  Mode   
call    3    -none- call   
K       1    -none- numeric
delta   2    -none- numeric
seed  626    -none- numeric

[1] 24.23151 24.23114

In [17]:
cv.error =rep(0,5) #initialize cv error with 0s
for(i in 1:5)
{
glm.fit =glm(mpg∼poly(horsepower,i),data=Auto) #polynomial models
cv.error[i]=cv.glm(Auto,glm.fit)$delta[1] #store the cv error of each model
}
cv.error

[1] 24.23151 19.24821 19.33498 19.42443 19.03321

it shows after 2nd order (i.e. qudratic model the CV error starts increasing )

## k-fold Cross-Validation

In [16]:
#5.3.3 k-Fold Cross-Validation
set.seed(17)
cv.error.10=rep(0,10) #using polynomial upto order 10
for(i in 1:10)
    {
    glm.fit=glm(mpg~poly(horsepower,i),data=Auto)
    cv.error.10[i]=cv.glm(Auto,glm.fit,K=10)$delta[1] #using 10-fold cross validation
}
cv.error.10

[1] 24.20520 19.18924 19.30662 19.33799 18.87911 19.02103 18.89609 19.71201
 [9] 18.95140 19.50196

Again after qudratic there does not seem to be very stark improvement in CV error

# Bootstrap

Estimating the accuracy of a statistic of interest
Two steps-
1. Create a function that computes statistic of interest 
2. Use boot() function part of boot library to perfrom bootstrap by repeatedly sampling observation from the data set with replacement

In [6]:

#5.3.4 The Bootstrap
#creating a function which takes as input the (X, Y ) data as well as a vector indicating which observations should be 
# used to estimate α. The function then outputs the estimate for α based on the selected observations
alpha.fn= function(data,index){
X= data$X[index]
Y= data$Y[index]
return((var(Y)-cov(X,Y))/(var(X)+var(Y)-2* cov(X,Y))) #statistic of interest
}

In [7]:
library(ISLR)
attach(Portfolio)
head(Portfolio)
alpha.fn(Portfolio,1:100) #taking first 100 data points for calculating alpha

The following objects are masked from Portfolio (pos = 3):

    X, Y

The following objects are masked from Portfolio (pos = 4):

    X, Y



X,Y
-0.8952509,-0.2349235
-1.5624543,-0.8851760
-0.4170899,0.2718880
1.0443557,-0.7341975
-0.3155684,0.8419834
-1.7371238,-2.0371910


[1] 0.5758321

In [4]:
set.seed(1)
alpha.fn(Portfolio, sample(100,100,replace=T)) #taking 100 random samples

[1] 0.5963833

In [24]:
#bootstrap analysis by performing this command many times
boot(Portfolio,alpha.fn,R=1000) #boot(data,statistic, sample)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original        bias    std. error
t1* 0.5758321 -7.315422e-05  0.08861826

### Estimating the accuracy of Linear regression Model
The bootstrap approach can be used to assess the variability of the coefficient estimates and predictions from a statistical learning method

In [10]:
# 1. creating a function for statistic which here is coeffiecients of linear regression
boot.fn= function (data ,index )
return (coef(lm(mpg∼horsepower,data=data , subset = index )))
boot.fn(Auto ,1:392) #taking sample from 1:392 to caculate statistic
    head(Auto)
set.seed (1)
boot.fn(Auto ,sample (392 ,392 , replace =T))  #bootstarp estimate of coefficients
    library(boot)
boot(Auto ,boot.fn ,1000) #1000 bootstrap estimates

(Intercept)  horsepower 
 39.9358610  -0.1578447

mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
18,8,307,130,3504,12.0,70,1,chevrolet chevelle malibu
15,8,350,165,3693,11.5,70,1,buick skylark 320
18,8,318,150,3436,11.0,70,1,plymouth satellite
16,8,304,150,3433,12.0,70,1,amc rebel sst
17,8,302,140,3449,10.5,70,1,ford torino
15,8,429,198,4341,10.0,70,1,ford galaxie 500


(Intercept)  horsepower 
 38.7387134  -0.1481952


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610  0.0296667441 0.860440524
t2* -0.1578447 -0.0003113047 0.007411218

In [11]:
summary(lm(mpg∼horsepower,data =Auto))$coef # finiding satndard error 


,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),39.9358610,0.717498656,55.65984,1.220362e-187
horsepower,-0.1578447,0.006445501,-24.48914,7.031989e-81


Bootstrap gives more accurate estimate of standard error tha the summary function

In [15]:
boot.fn= function (data ,index )
coefficients(lm(mpg∼horsepower+I(horsepower^2) ,data=data ,subset = index ))
set.seed (1)
boot(Auto,boot.fn ,1000)
summary(lm(mpg∼horsepower+I(horsepower^2) ,data=Auto ))$coef


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
        original        bias     std. error
t1* 56.900099702  6.098115e-03 2.0944855842
t2* -0.466189630 -1.777108e-04 0.0334123802
t3*  0.001230536  1.324315e-06 0.0001208339

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),56.900099702,1.8004268063,31.60367,1.740911e-109
horsepower,-0.466189630,0.0311246171,-14.97816,2.289429e-40
I(horsepower^2),0.001230536,0.0001220759,10.08009,2.196340e-21
